In [4]:
import csv 
import numpy
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

data = None
new = []
old = []
covid=[]


with open('abstract_cleaned_per_year.csv', 'r') as f:
    data = csv.reader(f)
    for i, row in enumerate(data):
        if i == 0:
            continue
        text = row[1]
        year = row[2]

        if int(year) > 2016 and int(year) < 2020:
            new.append(text)
        elif int(year) <= 2016:
            old.append(text)
        elif int(year) >= 2020:
            covid.append(text)

covid = ' '.join(covid)
new = ' '.join(new)
old = ' '.join(old)


vector_size = 50
window = 5
min_count = 1

models = []
for corpus in (old, new, covid):
    print('training...')
    models.append(
        Word2Vec(
            [simple_preprocess(corpus)], 
            vector_size=vector_size, 
            window=window, 
            min_count=min_count, 
            sg=1, 
            epochs=100
        )
    )

training...
training...
training...


Maybe this is a bit bleak, but I like to sanity check newly-trained embeddings with the word "women." If the embedding worked, it should come up with a bunch of gender stereotypes.

In [27]:
examine_concept("women").head(n=50)

,pre-2017,2017-2019,2020+
0,nonimmigrant,purposively,heterosexual
1,origin,agyw,ever
2,service,yearsold,dominant
3,turkish,nurse,cervical
4,significantly,exploratory,smw
5,single,partnership,every
6,muğla,girls,screening
7,stagnated,reaching,half
8,maternity,adolescent,invasive
9,acceptability,husbands,barrier


In [20]:
import pandas as pd
import tabulate

def examine_concept(concept):
    df = pd.DataFrame()
    for i, model in enumerate(models):
        df[str(i)] = [v[0] for v in model.wv.most_similar(concept, topn=100)]
    
    df.columns = ['pre-2017', '2017-2019', '2020+']
    return df 
    
examine_concept("misinformation").head(n=50)

,pre-2017,2017-2019,2020+
0,delays,out,spread
1,encountering,not,facilitating
2,short,act,intensifying
3,length,danger,selective
4,spontaneously,spreading,conspiracies
5,induce,ecker,rise
6,contradictory,mind,impressionable
7,heterogeneous,overhauls,spreading
8,increases,date,giving
9,delay,accidents,campaigns


This is promising considering how little data is in those abstracts. I'm now going to deploy